In [1]:
import re
import numpy as np
from functools import reduce
from collections import defaultdict
from string import ascii_uppercase, ascii_lowercase

def find_nums(s, neg = False):
    if neg:
        return re.findall(r'-?\d+', s)
    return re.findall(r'\d+', s)

In [3]:
# 1

def count_fuel(weight):
    weight = weight // 3 - 2
    if weight > 0: return weight + count_fuel(weight)
    return 0

def fueling(data):
    weights = [max((int(d) // 3) - 2, 0) for d in data]
    fuels = sum([count_fuel(x) for x in weights])
    print(sum(weights), sum(weights) + fuels)

with open('1') as f:
    fueling(f.read().split('\n'))


3252897 4876469


In [77]:
# 2

def opcodes(d, p1=True):
    for x in range(100):
        for y in range(100):
            data = [*(map(int, d))]
            data[1] = x
            data[2] = y
            i = 0
            while True:
                if i > len(data) - 3:
                    op, f, s, res = data[i:i + 4] + data[:((i + 4) % len(data))]
                else:
                    op, f, s, res = data[i:i+4]
                newpos = int(res) % int(len(data))
                if op == 99:
                    if p1:
                        print(data[0])
                        return
                    else:
                        if data[0] == 19690720:
                            print(x, y)
                            return
                    break
                elif op == 1:
                    data[newpos] = data[f] + data[s]
                elif op == 2:
                    data[newpos] = data[f] * data[s]

                i = (i + 4) % len(data)

with open('2') as f:
    lines = f.read().split(',')
    opcodes(lines)
    opcodes(lines, False)

490699
64 21


In [118]:
# 3

def solve_3(lines):
    intersects = set([])
    points = [{},{}]
    dx = {'L': -1, 'R': 1, 'U': 0, 'D': 0}
    dy = {'L': 0, 'R': 0, 'U': 1, 'D': -1}
    for i, line in enumerate(lines):
        x=y=0
        start = (0, 0, 0)
        directions = line.split(',')
        curr_dist = 0
        for d in directions:
            length = int(find_nums(d)[0])
            di = d[0]
            for j in range(length):
                x += dx[di]
                y += dy[di]
                curr_dist += 1
                
                if (x, y) not in points[i]:
                    points[i][(x, y)] = curr_dist
                    if i == 1 and (x, y) in points[0]:
                        intersects.add((x, y, points[1][(x, y)] + points[0][(x, y)]))
                        
    print(sorted([abs(p[0]) + abs(p[1]) for p in intersects])[0])
    print(sorted([abs(p[2]) for p in intersects])[0])
                
with open('3') as f:
    lines = f.readlines()
    solve_3(lines)
    

1017
11432
